In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
mpl.style.use('seaborn')
from matplotlib import pyplot as plt


from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report



In [14]:
# Load the data from the csv files

train_data = pd.read_csv("preprocessed_train_4b.csv", index_col="index")
test_data = pd.read_csv("preprocessed_test_4b.csv", index_col="index") 

In [4]:
target = "change_type"
train_data[target] = train_data[target].map({'Demolition': 0, 'Road': 1, 'Residential': 2, 'Commercial': 3, 'Industrial': 4,
       'Mega Projects': 5})

In [13]:
np.shape(train_data)

(310006, 47)

<h1> Model

In [5]:
import pandas as pd
import time
import numpy as np
import lightgbm as lgb
import gc

def lgb_modelfit_nocv(params, dtrain, dvalid, predictors, target='target', objective='binary', metrics='auc',
                 feval=None, early_stopping_rounds=20, num_boost_round=3000, verbose_eval=10, categorical_features=None):
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': objective,
        'metric':metrics,
        'learning_rate': 0.01,
        #'is_unbalance': 'true',  #because training data is unbalance (replaced with scale_pos_weight)
        'num_leaves': 31,  # we should let it be smaller than 2^(max_depth)
        'max_depth': -1,  # -1 means no limit
        'min_child_samples': 20,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 255,  # Number of bucketed bin for feature values
        'subsample': 0.6,  # Subsample ratio of the training instance.
        'subsample_freq': 0,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.3,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 5,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 0,  # L1 regularization term on weights
        'reg_lambda': 0,  # L2 regularization term on weights
        'nthread': 4,
        'verbose': 0,
        'metric':metrics
    }

    lgb_params.update(params)

    print("preparing validation datasets")

    xgtrain = lgb.Dataset(dtrain[predictors].values, label=dtrain[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )
    xgvalid = lgb.Dataset(dvalid[predictors].values, label=dvalid[target].values,
                          feature_name=predictors,
                          categorical_feature=categorical_features
                          )

    evals_results = {}

    bst1 = lgb.train(lgb_params, 
                     xgtrain, 
                     valid_sets=[xgtrain, xgvalid], 
                     valid_names=['train','valid'], 
                     evals_result=evals_results, 
                     num_boost_round=num_boost_round,
                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=10, 
                     feval=feval)

    n_estimators = bst1.best_iteration
    print("\nModel Report")
    print("n_estimators : ", n_estimators)
    print(metrics+":", evals_results['valid'][metrics][n_estimators-1])

    return bst1


In [6]:
train_df =train_data.sample(frac=0.8,random_state=200) #random state is a seed value
val_df =train_data.drop(train_df.index)

In [7]:
target = "change_type"
predictors = (train_data.columns.to_list()).copy()
predictors.remove(target)
categorical = ["change_" + str(i+1) + "_to_" + str(i+2) for i in range(len(["change_status_date1", "change_status_date2", "change_status_date3", "change_status_date4", "change_status_date5"]) - 1)]
categorical += ["change_status_date1", "change_status_date2", "change_status_date3", "change_status_date4", "change_status_date5"]

In [8]:
train_df[predictors]

,change_status_date1,change_status_date2,change_status_date3,change_status_date4,change_status_date5,date1,date2,date3,date4,date5,...,perimetre,logPerimetre,aire,logAire,airePerimetreRatio,logAirePerimetreRatio,airePerimetre2Ratio,squareness,logSquareness,nVert
index,,,,,,,,,,,,,,,,,,,,,
39941,1,0,1,1,1,2086,153,563,832,913,...,0.000565,-7.477884,1.981428e-08,-17.736863,0.000035,-10.258979,0.061971,0.186594,-1.678821,5.0
213878,4,4,4,3,2,1533,365,905,1383,2029,...,0.000486,-7.628277,1.443459e-08,-18.053638,0.000030,-10.425362,0.060988,0.443646,-0.812728,5.0
108729,5,6,6,5,5,1841,596,992,1383,2039,...,0.000349,-7.959273,6.701612e-09,-18.820918,0.000019,-10.861645,0.054893,1.499993,0.405460,5.0
133269,5,6,7,7,0,1827,540,1036,1382,2083,...,0.000522,-7.558455,1.690910e-08,-17.895414,0.000032,-10.336959,0.062131,0.252672,-1.375664,5.0
131501,4,4,0,0,0,1827,540,1036,1382,2083,...,0.000647,-7.342405,2.158708e-08,-17.651171,0.000033,-10.308766,0.051490,1.614538,0.479049,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225327,1,1,6,6,2,1648,800,1182,1669,2203,...,0.000197,-8.529966,2.340585e-09,-19.872865,0.000012,-11.342899,0.060029,0.586743,-0.533168,5.0
76752,1,1,0,0,0,1648,201,688,1235,2066,...,0.000420,-7.774162,1.025069e-08,-18.395921,0.000024,-10.621759,0.057984,0.733308,-0.310189,5.0
192246,4,0,1,0,0,2097,394,770,1165,1505,...,0.001502,-6.500777,1.474839e-07,-15.729547,0.000098,-9.228770,0.065350,0.300326,-1.202886,7.0


In [9]:
params = {
    'learning_rate': 0.15,
    #'is_unbalance': 'true', # replaced with scale_pos_weight argument
    'num_leaves': 7,  # 2^max_depth - 1
    'max_depth': 3,  # -1 means no limit
    'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
    'max_bin': 100,  # Number of bucketed bin for feature values
    'subsample': 0.7,  # Subsample ratio of the training instance.
    'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
    'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.
    'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
    'scale_pos_weight':99, # because training data is extremely unbalanced 
    'num_class' : 6
}

In [10]:
bst = lgb_modelfit_nocv(params, 
                        train_df, 
                        val_df, 
                        predictors, 
                        target, 
                        objective='multiclass', 
                        metrics='multi_logloss',
                        early_stopping_rounds=30, 
                        verbose_eval=True, 
                        num_boost_round=500, 
                        categorical_features=categorical)

preparing validation datasets


C:\Users\jonat\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\jonat\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\jonat\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\jonat\AppData\Loca

[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.084822 seconds.
You can set `force_col_wise=true` to remove the overhead.


C:\Users\jonat\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\jonat\AppData\Local\Programs\Python\Python39\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
Training until validation scores don't improve for 30 rounds
[10]	train's multi_logloss: 0.700933	valid's multi_logloss: 0.701891
[20]	train's multi_logloss: 0.657005	valid's multi_logloss: 0.658833
[30]	train's multi_logloss: 0.638659	valid's multi_logloss: 0.641439
[40]	train's multi_logloss: 0.627006	valid's multi_logloss: 0.630799
[50]	train's multi_logloss: 0.617916	valid's multi_logloss: 0.622491
[60]	train's multi_logloss: 0.610521	valid's multi_logloss: 0.616355
[70]	train's multi_logloss: 0.604345	valid's multi_logloss: 0.611556
[80]	train's multi_logloss: 0.599728	valid's multi_logloss: 0.60811
[90]	train's multi_logloss: 0.595217	valid's multi_logloss: 0.604792
[100]	train's multi_logloss: 0.591681	valid's multi_logloss: 0.602261
[110]	train's multi_logloss: 0.588094	valid's multi_logloss: 0.599643
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

In [30]:
y_pred = np.argmax(bst.predict(val_df[predictors]), axis = 1)



In [33]:
for y in y_pred:
    print(y)

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
0
2
2
2
2
2
2
2
2
2
2
2
2
0
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
0
0
0
0
0
0
0
3
3
3
0
3
3
2
3
3
3
2
2
3
3
1
3
2
3
2
3
3
3
3
3
3
2
3
3
3
3
2
3
3
3
3
3
3
3
3
3
3
3
0
3
3
0
1
3
3
3
2
3
3
3
2
3
3
2
3
3
3
2
2
3
3
3
3
3
1
3
1
1
1
1
3
1
3
2
2
3
0
3
3
2
2
2
2
2
2
2
0
3
3
3
0
2
3
0
0
0
0
0
3
2
2
3
3
2
2
2
0
2
0
2
2
2
3
3
3
3
3
1
2
1
2
1
1
1
2
2
2
0
2
3
1
0
0
2
2
2
2
2
2
2
3
0
0
0
3
3
2
2
2
2
2
2
1
1
2
2
2
2
1
0
1
0
0
1
1
2
0
3
0
0
0
3
0
0
2
2
3
0
0
2
2
2
2
3
2
3
3
2
3
2


In [15]:
# For submission

y_pred_final = np.argmax(bst.predict(test_data), axis = 1)
print(y_pred_final.shape)

pred_df = pd.DataFrame(y_pred_final, columns=['change_type'])
pred_df.to_csv("submissions/desesp_sample_submission.csv", index=True, index_label='Id')

(121704,)
